In [13]:
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.layers.advanced_activations import *
from keras.datasets import mnist

from keras import initializers

random_dim = 100

In [14]:
(x_train, _), (_, _) = mnist.load_data()
x_train = x_train.astype('float64') / 255
x_train = x_train.reshape(60000, 784)

In [15]:
generator = Sequential()
generator.add(Dense(256, input_dim=random_dim))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(0.3))
discriminator.add(Dropout(0.3))

generator.add(Dense(units=512 ,activity_regularizer=regularizers.l2(0.01)))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(0.3))
discriminator.add(Dropout(0.3))

generator.add(Dense(units=1024, activity_regularizer=regularizers.l2(0.01)))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(0.3))
discriminator.add(Dropout(0.3))

generator.add(Dense(784, activation='tanh'))
generator.compile(loss='binary_crossentropy', optimizer='adam')

In [16]:
discriminator = Sequential()
discriminator.add(Dense(1024, input_dim=784))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(0.3))
discriminator.add(Dropout(0.3))

discriminator.add(Dense(units=512, activity_regularizer=regularizers.l2(0.01)))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(0.3))
discriminator.add(Dropout(0.3))

discriminator.add(Dense(units=256, activity_regularizer=regularizers.l2(0.01)))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(0.3))
discriminator.add(Dropout(0.3))

discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

In [17]:
discriminator.trainable = False
gan_input = Input(shape=(random_dim,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(inputs=gan_input, outputs=gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [18]:
%pylab inline
import matplotlib.pyplot as plt

def plot_generated_images(epoch, generator, examples=25, dim=(5, 5), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('genrerated images simple/image_epoch_%d.png' % epoch)

Populating the interactive namespace from numpy and matplotlib


In [19]:
def plot_loss(losses_d, losses_g, epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(losses_d, label='Discriminitive loss')
    plt.plot(losses_g, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

In [ ]:
from tqdm import tqdm
epochs = 500
batch_size = 128

# Split the training data into batches of size 128
batch_count = int(x_train.shape[0] / batch_size)

losses_d = []
losses_g = []

for e in range(epochs):
    print('-'*15, 'Epoch %d' % e, '-'*15)
    for _ in tqdm(range(batch_count)):
        # Get a random set of input noise and images
        noise = np.random.normal(0, 1, size=[batch_size, random_dim])
        image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

        # Generate fake MNIST images
        generated_images = generator.predict(noise)
        X = np.concatenate([image_batch, generated_images])

        # Labels for generated and real data
        y_dis = np.zeros(2*batch_size)
        # One-sided label smoothing
        y_dis[:batch_size] = 0.9

        # Train discriminator
        discriminator.trainable = True
        d_loss = discriminator.train_on_batch(X, y_dis)

        # Train generator
        noise = np.random.normal(0, 1, size=[batch_size, random_dim])
        y_gen = np.ones(batch_size)
        discriminator.trainable = False
        g_loss = gan.train_on_batch(noise, y_gen)
        
        losses_g.append(g_loss)
        losses_d.append(d_loss)
        
        
    # Plot dat shit:
    if e % 25 == 0:
        plot_generated_images(e, generator)
        
    plot_loss(losses_d, losses_g, e)

--------------- Epoch 0 ---------------


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [01:28<00:00,  3.73it/s]


--------------- Epoch 1 ---------------


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [02:06<00:00,  3.97it/s]


--------------- Epoch 2 ---------------


...

100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [02:38<00:00,  2.95it/s]


--------------- Epoch 436 ---------------


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [02:33<00:00,  2.92it/s]


--------------- Epoch 437 ---------------


 41%|████████████████████████████████▍                                               | 190/468 [01:04<01:50,  2.51it/s]